In [ ]:
import os
from dotenv import load_dotenv


load_dotenv()

API_KEY = os.getenv("GEMINI_API_KEY") 
print(API_KEY)

In [ ]:
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
# Define the state for our graph
class AgentState(TypedDict):
    """The state of the agent."""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    agent_type: str  # Will store which agent was selected

In [ ]:
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.7,
    google_api_key=API_KEY
)

In [ ]:
# Router function - decides which agent to use based on the first message
def route_to_agent(state: AgentState) -> str:
    """
    Analyzes the first user message and decides whether to route to therapist or joker.
    """
    # Get the first user message
    user_message = state["messages"][0].content
    
    # Use LLM to decide which agent to use
    routing_prompt = f"""Based on the following user message, decide if the user needs:
- A THERAPIST (for emotional support, mental health, serious problems, advice)
- A JOKER (for jokes, humor, entertainment, fun)

User message: "{user_message}"

Respond with ONLY one word: either "therapist" or "joker"."""
    
    response = llm.invoke(routing_prompt)
    decision = response.content.strip().lower()
    
    # Return the agent type
    if "therapist" in decision:
        return "therapist"
    else:
        return "joker"

In [ ]:
# Therapist Node
def therapist_node(state: AgentState):
    """
    Acts as a compassionate therapist providing emotional support.
    """
    messages = state["messages"]
    
    # Create therapist system prompt
    therapist_prompt = """You are a compassionate and professional therapist. 
Your role is to:
- Listen actively and empathetically
- Provide emotional support and validation
- Ask thoughtful questions to understand the person better
- Offer helpful coping strategies when appropriate
- Maintain a warm, caring, and professional tone

Respond to the user's message with empathy and understanding."""
    
    # Combine system prompt with user messages
    full_messages = [HumanMessage(content=therapist_prompt)] + list(messages)
    
    # Get response from LLM
    response = llm.invoke(full_messages)
    
    return {
        "messages": [response],
        "agent_type": "therapist"
    }

In [ ]:
# Joker Node
def joker_node(state: AgentState):
    """
    Acts as a funny comedian providing jokes and humor.
    """
    messages = state["messages"]
    
    # Create joker system prompt
    joker_prompt = """You are a hilarious comedian and joker.
Your role is to:
- Make people laugh with clever jokes and witty responses
- Use wordplay, puns, and humor
- Keep things light and entertaining
- Be creative and spontaneous with your comedy
- Stay positive and fun

Respond to the user's message with humor and jokes."""
    
    # Combine system prompt with user messages
    full_messages = [HumanMessage(content=joker_prompt)] + list(messages)
    
    # Get response from LLM
    response = llm.invoke(full_messages)
    
    return {
        "messages": [response],
        "agent_type": "joker"
    }

In [ ]:
# Build the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("therapist", therapist_node)
workflow.add_node("joker", joker_node)

# Set entry point with conditional routing
workflow.add_conditional_edges(
    START,
    route_to_agent,
    {
        "therapist": "therapist",
        "joker": "joker"
    }
)

# Add edges to END after each agent responds
workflow.add_edge("therapist", END)
workflow.add_edge("joker", END)

# Compile the graph
graph = workflow.compile()

In [ ]:
# Visualize the graph (optional)
try:
    from IPython.display import Image, display
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not visualize graph: {e}")

In [ ]:
# Test 1: Message that should route to therapist
print("=" * 50)
print("TEST 1: Routing to Therapist")
print("=" * 50)

initial_state_therapist = {
    "messages": [HumanMessage(content="I'm feeling really stressed about work and life lately")],
    "agent_type": ""
}

result_therapist = graph.invoke(initial_state_therapist)
print(f"\nAgent selected: {result_therapist['agent_type']}")
print(f"\nResponse: {result_therapist['messages'][-1].content}")

In [ ]:
# Test 2: Message that should route to joker
print("\n" + "=" * 50)
print("TEST 2: Routing to Joker")
print("=" * 50)

initial_state_joker = {
    "messages": [HumanMessage(content="Tell me a funny joke to brighten my day!")],
    "agent_type": ""
}

result_joker = graph.invoke(initial_state_joker)
print(f"\nAgent selected: {result_joker['agent_type']}")
print(f"\nResponse: {result_joker['messages'][-1].content}")

## How This Works

This LangGraph implementation demonstrates:

1. **State Management**: Uses `AgentState` to track messages and agent type
2. **Conditional Routing**: The `route_to_agent` function uses Gemini LLM to analyze the first message and decide which agent to invoke
3. **Two Agent Nodes**:
   - **Therapist Node**: Provides empathetic emotional support
   - **Joker Node**: Delivers humor and entertainment
4. **Graph Flow**: 
   - START → Router (conditional) → Therapist OR Joker → END

The LLM intelligently routes based on the user's intent detected in their first message.